In [60]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from fastai.imports import *
from fastai.core import *
from fastai.io import *
from fastai.dataloader import *
from fastai.conv_learner import *
from fastai.learner import *
import os
from audio_dataset import *
from audio_transforms import *

import IPython.display as ipd

In [61]:
import librosa
from librosa import display

## Load Data

In [62]:
PATH = Path('data/freesound')
TRN_PATH = PATH/'audio_train'
TEST_PATH = PATH/'audio_test'

In [63]:
list(PATH.iterdir())

[PosixPath('data/freesound/sample_submission.csv'),
 PosixPath('data/freesound/models'),
 PosixPath('data/freesound/train.csv'),
 PosixPath('data/freesound/audio_train'),
 PosixPath('data/freesound/audio_test'),
 PosixPath('data/freesound/tmp')]

In [64]:
trn = pd.read_csv(PATH/'train.csv')
test = pd.read_csv(PATH/'sample_submission.csv')

In [65]:
trn.head()

fname         label  manually_verified
0  00044347.wav        Hi-hat                  0
1  001ca53d.wav     Saxophone                  1
2  002d256b.wav       Trumpet                  0
3  0033e230.wav  Glockenspiel                  1
4  00353774.wav         Cello                  1

In [66]:
verified = list(trn['manually_verified'])

In [67]:
trn = trn[['fname','label']].copy()
trn_sample = trn[:200]
trn.head()

fname         label
0  00044347.wav        Hi-hat
1  001ca53d.wav     Saxophone
2  002d256b.wav       Trumpet
3  0033e230.wav  Glockenspiel
4  00353774.wav         Cello

In [68]:
trn.to_csv('trn.csv', index=False)
trn_sample.to_csv('trn_sample.csv', index=False)

In [69]:
test.head()

fname                  label
0  00063640.wav  Laughter Hi-Hat Flute
1  0013a1db.wav  Laughter Hi-Hat Flute
2  002bb878.wav  Laughter Hi-Hat Flute
3  002d392d.wav  Laughter Hi-Hat Flute
4  00326aa9.wav  Laughter Hi-Hat Flute

In [70]:
trn.shape[0], len(trn.label.unique())

(9473, 41)

In [71]:
fnames = list(trn['fname']) 
len(fnames), fnames[:5]

(9473,
 ['00044347.wav',
  '001ca53d.wav',
  '002d256b.wav',
  '0033e230.wav',
  '00353774.wav'])

In [72]:
test_fnames = list(test['fname']) 
len(test_fnames), test_fnames[:5]

(9400,
 ['00063640.wav',
  '0013a1db.wav',
  '002bb878.wav',
  '002d392d.wav',
  '00326aa9.wav'])

In [108]:
trn_wavs = (PATH/'audio_train').glob('*.wav')
test_wavs = (PATH/'audio_test').glob('*.wav')

In [122]:
x,sr = librosa.load(next(trn_wavs), sr=None); x.shape

(264600,)

In [116]:
x = librosa.effects.time_stretch(x, 2.0); x.shape

(69120,)

In [121]:
x = librosa.effects.pitch_shift(x, sr, n_steps=4); x.shape

(754992,)

In [117]:
aud = get_audio(next(trn_wavs), melspect=True, feature_name='log_mel_spec'); aud.shape

(259, 128)

In [123]:
aud = librosa.stft(x, n_fft=1024, hop_length=512)

In [125]:
aud_fast = librosa.phase_vocoder(aud, 2.0).T; aud_fast.shape

(259, 513)

In [104]:
aud_arr = np.array([get_audio(o, melspect=True, feature_name='log_mel_spec') for o in list(trn_wavs)])

In [105]:
np.save(PATH/'tmp/aud_arr_10s.npy', aud_arr)

In [35]:
aud_arr = np.load(PATH/'tmp/aud_arr_10s.npy'); aud_arr.shape

In [106]:
mean = np.mean(aud_arr)
std = np.std(aud_arr)
stats = (np.array([mean]), np.array([std]))
mean, std

(0.1961404119699115, 0.32234817744998295)

In [79]:
stats = (np.array([mean]), np.array([std]))

### Distribution of Categories

In [ ]:
cat_grp = trn.groupby(['label', 'manually_verified']).count()
plot = cat_grp.unstack().reindex(cat_grp.unstack().sum(axis=1).sort_values().index).plot(
    kind='bar', stacked=True, title='Number of Audio Samples per Category', figsize=(16,10))
plot.set_xlabel('Category')
plot.set_ylabel('Number of Samples');

Fairly small percentage of "manually verified" sound clips.

In [ ]:
print('Minimum samples in a category:', min(trn.label.value_counts()))
print('Maximum samples in a category:', max(trn.label.value_counts()))

In [ ]:
import wave

f = str(PATH/'audio_train/001ca53d.wav')
wav = wave.open(f)
print('Sampling (frame) rate:', wav.getframerate())
print('Total samples (frames):', wav.getnframes())
print('Duration:', wav.getnframes()/wav.getframerate())

In [ ]:
#import torchaudio -- torchaudio not available in Fastai environment
# using scipy
from scipy.io import wavfile
rate, data = wavfile.read(f)
print('Sampling (frame) rate:', rate)
print('Total samples (frames):', data.shape)
print(data)

Plot audio frames

In [ ]:
plt.plot(data, '-', );

Zoom in to view 250 frames

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(data[25250:25500], '.'); plt.plot(data[25250:25500], '-');

In [ ]:
trn['nframes'] = trn['fname'].apply(lambda f: wave.open('data/freesound/audio_train/'+f).getnframes())
test['nframes'] = test['fname'].apply(lambda f: wave.open('data/freesound/audio_test/'+f).getnframes())

_,ax = plt.subplots(figsize=(16,4))
sns.violinplot(ax=ax, x='label', y='nframes', data=trn)
plt.xticks(rotation=90)
plt.title('Distribution of audio frames, per label', fontsize=16)
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,5))
trn.nframes.hist(bins=100, ax=axes[0])
test.nframes.hist(bins=100, ax=axes[1])
plt.suptitle("Frame Length Distribution in Train and Test", ha='center', fontsize='large');

1. The majority of frame lengths are short.
2. There are 4 abnormal lengths in test set. Let's analyze them.

### Listen to Sounds

In [48]:
[preds[i] for i in idx[:10]]

['Fireworks Finger_snapping Microwave_oven',
 'Oboe Flute Trumpet',
 'Bass_drum Knock Fireworks',
 'Double_bass Bass_drum Knock',
 'Oboe Electric_piano Telephone',
 'Bass_drum Knock Double_bass',
 'Squeak Violin_or_fiddle Harmonica',
 'Gong Acoustic_guitar Chime',
 'Flute Telephone Clarinet',
 'Saxophone Flute Clarinet']

In [59]:
#play sample with stats
n = 11
#length = 3*44100
sample = os.path.join(TRN_PATH, fnames[n])
print(test_fnames[n])
raw = open_audio(sample)
raw_len = len(raw)
raw_s = adj_length(raw)
print('raw length: ', raw_len, 'sample length:', len(raw_s))
print('label:', trn['label'].iloc[n], 'verified:', verified[n])
ipd.Audio(raw_s, rate=44100)

00a7a2f6.wav
raw length:  132300 sample length: 441000
label: Keys_jangling verified: 0


### Audio_transforms

In [75]:
class Transform():
    """ A class that represents a transform.

    All other transforms should subclass it. All subclasses should override
    do_transform.

    Arguments
    ---------
        tfm_y : TfmType
            type of transform
    """
    def __init__(self, tfm_y=TfmType.NO):
        self.tfm_y=tfm_y
        self.store = threading.local()

    def set_state(self): pass
    def __call__(self, x, y):
        self.set_state()
        x,y = ((self.transform(x),y) if self.tfm_y==TfmType.NO
                else self.transform(x,y) if self.tfm_y in (TfmType.PIXEL, TfmType.CLASS)
                else self.transform_coord(x,y))
        return x, y

    def transform_coord(self, x, y): return self.transform(x),y

    def transform(self, x, y=None):
        x = self.do_transform(x,False)
        return (x, self.do_transform(y,True)) if y is not None else x

    @abstractmethod
    def do_transform(self, x, is_y): raise NotImplementedError


class Denormalize():
    """ De-normalizes an image, returning it to original format.
    """
    def __init__(self, m, s):
        self.m=np.array(m, dtype=np.float32)
        self.s=np.array(s, dtype=np.float32)
    def __call__(self, x): return x*self.s+self.m


class Normalize():
    """ Normalizes an image to zero mean and unit standard deviation, given the mean m and std s of the original image """
    def __init__(self, m, s): #tfm_y=TfmType.NO
        self.m=np.array(m, dtype=np.float32)
        self.s=np.array(s, dtype=np.float32)
        #self.tfm_y=tfm_y

    def __call__(self, x, y=None):
        x = (x-self.m)/self.s
        #if self.tfm_y==TfmType.PIXEL and y is not None: y = (y-self.m)/self.s
        return x,y

def get_mfcc(s,y):
    x = librosa.feature.mfcc(S=librosa.power_to_db(s[0,:,:]))
    x = x.reshape(1,x.shape[0],x.shape[1])
    return x, y

def log_mel(s,y):
    x = librosa.amplitude_to_db(s[0,:,:])
    x = x + .001
    x = x.reshape(1,x.shape[0],x.shape[1])
    return x, y
       
def compose(im, y, fns):
    """ apply a collection of transformation functions fns to images
    """
    for fn in fns:
        #pdb.set_trace()
        im, y =fn(im, y)
    return im if y is None else (im, y)


class Transforms():
    def __init__(self, sz, tfms, normalizer, denorm, crop_type=CropType.CENTER,
                 tfm_y=TfmType.NO, sz_y=None):
        if sz_y is None: sz_y = sz
        self.sz,self.denorm,self.norm,self.sz_y = sz,denorm,normalizer,sz_y
        crop_tfm = crop_fn_lu[crop_type](sz, tfm_y, sz_y)
        self.tfms = tfms
        self.tfms.append(crop_tfm)
        if normalizer is not None: self.tfms.append(normalizer)
        self.tfms.append(ChannelOrder(tfm_y))

    def __call__(self, im, y=None): return compose(im, y, self.tfms)
    def __repr__(self): return str(self.tfms)


def image_gen(normalizer, denorm, sz, tfms=None, max_zoom=None, pad=0, crop_type=None,
              tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, scale=None):
    """
    Generate a standard set of transformations

    Arguments
    ---------
     normalizer :
         image normalizing function
     denorm :
         image denormalizing function
     sz :
         size, sz_y = sz if not specified.
     tfms :
         iterable collection of transformation functions
     max_zoom : float,
         maximum zoom
     pad : int,
         padding on top, left, right and bottom
     crop_type :
         crop type
     tfm_y :
         y axis specific transformations
     sz_y :
         y size, height
     pad_mode :
         cv2 padding style: repeat, reflect, etc.

    Returns
    -------
     type : ``Transforms``
         transformer for specified image operations.

    See Also
    --------
     Transforms: the transformer object returned by this function
    """
    if tfm_y is None: tfm_y=TfmType.NO
    if tfms is None: tfms=[]
    elif not isinstance(tfms, collections.Iterable): tfms=[tfms]
    if sz_y is None: sz_y = sz
    if scale is None:
        scale = [RandomScale(sz, max_zoom, tfm_y=tfm_y, sz_y=sz_y) if max_zoom is not None
                 else Scale(sz, tfm_y, sz_y=sz_y)]
    elif not is_listy(scale): scale = [scale]
    if pad: scale.append(AddPadding(pad, mode=pad_mode))
    if crop_type!=CropType.GOOGLENET: tfms=scale+tfms
    return Transforms(sz, tfms, normalizer, denorm, crop_type,
                      tfm_y=tfm_y, sz_y=sz_y)

def noop(x):
    """dummy function for do-nothing.
    equivalent to: lambda x: x"""
    return x

class AudTransforms():
    def __init__(self, tfms, normalizer, denorm):
        #if sz_y is None: sz_y = sz
        #self.sz,self.denorm,self.norm,self.sz_y = sz,denorm,normalizer,sz_y
        self.denorm,self.norm = denorm,normalizer
        #crop_tfm = crop_fn_lu[crop_type](sz, tfm_y, sz_y)
        self.tfms = tfms
        #self.tfms.append(crop_tfm)
        if normalizer is not None: self.tfms.append(normalizer)
        #self.tfms.append(ChannelOrder(tfm_y))

    def __call__(self, im, y=None): return compose(im, y, self.tfms) 
    def __repr__(self): return str(self.tfms)

def audio_gen(normalizer, denorm, tfms=None):
    if tfms is None: tfms = []
    elif not isinstance(tfms, collections.Iterable): tfms=[tfms]
    return AudTransforms(tfms, normalizer, denorm)

def aud_tfms_from_stats(stats, aug_tfms=None):
#def tfms_from_stats(stats, sz, aug_tfms=None, max_zoom=None, pad=0, crop_type=CropType.RANDOM,
                    #tfm_y=None, sz_y=None, pad_mode=cv2.BORDER_REFLECT, norm_y=True, scale=None):
    """ Given the statistics of the training image sets, returns separate training and validation transform functions
    """
    #if aug_tfms is None: aug_tfms=[]
    #tfm_norm = Normalize(*stats, tfm_y=tfm_y if norm_y else TfmType.NO) if stats is not None else None
    tfm_norm = Normalize(*stats)
    tfm_denorm = Denormalize(*stats) if stats is not None else None
    #val_crop = CropType.CENTER if crop_type in (CropType.RANDOM,CropType.GOOGLENET) else crop_type
    #val_tfm = image_gen(tfm_norm, tfm_denorm, sz, pad=pad, crop_type=val_crop,
            #tfm_y=tfm_y, sz_y=sz_y, scale=scale)
    val_tfm = audio_gen(tfm_norm, tfm_denorm, tfms=aug_tfms)
    #trn_tfm = image_gen(tfm_norm, tfm_denorm, sz, pad=pad, crop_type=crop_type,
            #tfm_y=tfm_y, sz_y=sz_y, tfms=aug_tfms, max_zoom=max_zoom, pad_mode=pad_mode, scale=scale)
    trn_tfm = audio_gen(tfm_norm, tfm_denorm, tfms=aug_tfms)
    return trn_tfm, val_tfm


### ResNet

In [76]:
def conv(ni, nf, ks=3, stride=1):
    return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=False)


def bn1(planes):
    m = nn.BatchNorm1d(planes)
    m.weight.data.fill_(1)
    m.bias.data.zero_()
    return m

def bn(planes, init_zero=False):
    m = nn.BatchNorm2d(planes)
    m.weight.data.fill_(0 if init_zero else 1)
    m.bias.data.zero_()
    return m

class fc1(nn.Module):
    def __init__(self, ni, nf, ks=2, stride=1, padding=1):
        super().__init__()
        self.conv = nn.Conv2d(ni,nf,kernel_size=ks,stride=stride, padding=padding)
        self.relu = nn.ReLU(inplace=True)
        self.max = nn.MaxPool2d(2, stride=2, padding=1)
    
    def forward(self,x):
        out = self.conv(x)
        #return self.relu(out)
        out = self.relu(out)
        out = self.max(out)
        return out

class fc2(nn.Module):
    def __init__(self, ni, nf, ks=1, stride=1):
        super().__init__()
        self.conv = nn.Conv2d(ni,nf,kernel_size=ks,stride=stride)
        #self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self,x):
        out = self.conv(x)
        #return self.sigmoid(out)
        return self.relu(out)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = conv(inplanes, planes, stride=stride)
        self.bn1 = bn(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv(planes, planes)
        self.bn2 = bn(planes)
        self.downsample = downsample
        self.stride = stride
        #self.max = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        #self.drop = nn.Dropout2d(0.1)

    def forward(self, x):
        residual = x
        if self.downsample is not None: residual = self.downsample(x)

        out = self.conv1(x)
        out = self.relu(out)
        out = self.bn1(out)

        out = self.conv2(out)

        out = residual + out
        out = self.relu(out)
        out = self.bn2(out)
        
        #out = self.max(out)
        #out = self.drop(out)

        return out
    

class MyResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000, k=1, vgg_head=False):
        super().__init__()
        self.inplanes = 64

        features = [conv(1, 64, ks=3, stride=2)
            , bn(64) , nn.ReLU(inplace=True) , nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
            , self._make_layer(block, int(64*k), layers[0])
            , self._make_layer(block, int(128*k), layers[1], stride=2)
            , self._make_layer(block, int(256*k), layers[2], stride=2)
            , self._make_layer(block, int(512*k), layers[3], stride=2)]
        out_sz = int(512*k) * block.expansion

        if vgg_head:
            features += [nn.AdaptiveAvgPool2d(3), Flatten()
                , nn.Linear(out_sz*3*3, 4096), nn.ReLU(inplace=True), bn1(4096), nn.Dropout(0.25)
                , nn.Linear(4096,   4096), nn.ReLU(inplace=True), bn1(4096), nn.Dropout(0.25)
                , nn.Linear(4096, num_classes)]
        else: features += [nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(out_sz, num_classes)]
        #else: features += [nn.AdaptiveMaxPool2d(3), fc1(out_sz, 1024), fc2(1024, 41), nn.AdaptiveAvgPool2d(1),
                           #Flatten(), nn.Linear(41, num_classes)]

        self.features = nn.Sequential(*features)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv(self.inplanes, planes*block.expansion, ks=1, stride=stride),
                bn(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks): layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)#, nn.Dropout2d(0.5))

    def forward(self, x): return self.features(x)


In [77]:
# from John
def mapk_np(preds, targs, k=3):
    preds = np.argsort(-preds, axis=1)[:, :k]
    score = 0.0
    for i in range(k):
        num_hits = (preds[:, i] == targs).sum()
        score += num_hits * (1.0 / (i+1.0))
    score /= preds.shape[0]
    return score

def mapk(preds, targs, k=3):
    return mapk_np(to_np(preds), to_np(targs), k)

### Model

In [80]:
tfms = aud_tfms_from_stats(stats, aug_tfms=None)
md = AudioClassifierData.from_csv(PATH, 'audio_train', 'trn_sample.csv', tfms=tfms, test_name='audio_test')

In [70]:
m = MyResNet(BasicBlock, [3, 4, 6, 3], num_classes=41, vgg_head=False)
#model = BasicModel(to_gpu(m), name='ResNet')
opt = optim.Adam
metrics = [accuracy, mapk]
loss = F.cross_entropy
learn = ConvLearner.from_model_data(m, md, crit=loss, metrics=metrics, opt_fn=opt)

In [71]:
learn.unfreeze()

In [60]:
learn.lr_find()

 82%|████████▏ | 97/119 [04:10<00:56,  2.58s/it, loss=14.5]

In [61]:
learn.sched.plot()

In [62]:
learn.fit(1e-3, 1, wds=1e-5, cycle_len=5, use_clr_beta=(5,20,0.95,0.75))

epoch      trn_loss   val_loss   accuracy   mapk            
    0      2.910675   3.006009   0.208025   0.301126  
    1      2.489756   3.716091   0.229673   0.32735         
    2      2.135787   2.062081   0.417635   0.527367        
    3      1.750009   1.825572   0.5        0.592397        
    4      1.451959   1.593763   0.55755    0.647395        


[array([ 1.59376]), 0.5575501587725743, 0.64739528335093266]

In [63]:
learn.save('2d_resnet_trim_adj')

In [27]:
learn.load('2d_resnet_aug')

In [64]:
learn.fit(1e-3, 1, wds=1e-5, cycle_len=5, use_clr_beta=(5,20,0.95,0.75))

epoch      trn_loss   val_loss   accuracy   mapk            
    0      1.824353   3.410902   0.265048   0.346357  
    1      1.919786   2.769285   0.337381   0.433298        
    2      1.708295   1.931068   0.456177   0.563534        
    3      1.382881   1.513866   0.577614   0.669395        
    4      1.118543   1.409156   0.605597   0.69201         


[array([ 1.40916]), 0.60559662078224996, 0.69200985568461792]

In [65]:
learn.save('2d_resnet_trim_adj_2')

In [146]:
learn.load('resnet_2_connected')

In [66]:
learn.fit(1e-3, 1, wds=1e-5, cycle_len=5, use_clr_beta=(5,20,0.95,0.75))

epoch      trn_loss   val_loss   accuracy   mapk            
    0      1.333573   2.444052   0.399155   0.501408  
    1      1.677367   2.170311   0.407603   0.51144         
    2      1.409418   2.532168   0.378564   0.470961        
    3      1.118843   1.414954   0.610876   0.695882        
    4      0.835637   1.342595   0.628828   0.712073         


[array([ 1.34259]), 0.62882787706733634, 0.71207321365716314]

In [67]:
learn.save('2d_resnet_trim_adj_3')

In [72]:
learn.load('2d_resnet_trim_adj_3')

In [68]:
learn.freeze()

In [69]:
learn.fit(1e-3, 3, wds=1e-5)

epoch      trn_loss   val_loss   accuracy   mapk            
    0      1.475328   1.930132   0.492608   0.593013  
    1      1.391155   1.883881   0.501056   0.592397        
    2      1.327014   1.829744   0.515312   0.608765        


[array([ 1.82974]), 0.51531151003167897, 0.60876451953537492]

In [73]:
learn.model.eval()
val_preds = learn.predict_with_targs()

val_acc = accuracy_np(*val_preds)
val_map = mapk_np(*val_preds)

print(f'Val Acc: {val_acc:.3f}, Val MAP: {val_map:.3f}')

Val Acc: 0.629, Val MAP: 0.712


### Predictions

In [74]:
multi_preds, y = learn.TTA(is_test=True)

OSError: Error handling audio at: data/freesound/audio_test/b39975f5.wav

In [75]:
%debug

> /home/paperspace/deeplearning/me-FastAudio/scratch/audio_dataset.py(63)open_audio()
     61             return aud#, sr#, l
     62         except Exception as e:
---> 63             raise OSError('Error handling audio at: {}'.format(fn)) from e
     64 
     65 # returns raw or aelspectrogram if melspect=True

ipdb> aud.shape
(0,)
ipdb> q


In [35]:
preds = np.mean(multi_preds, 0)

In [36]:
np.save(PATH/'tmp/preds_6.npy', preds)

In [65]:
preds = np.load(PATH/'tmp/preds_6.npy')

In [43]:
top_3_idx = [np.argsort(preds[i])[-3:][::-1] for i in range(len(test_fnames))]
classes = np.array(sorted(trn.label.unique()))
pred_labels = [list(classes[[top_3_idx[i]]]) for i in range(len(test_fnames))]
preds = [" ".join(ls) for ls in pred_labels]
preds[:5]

array(['Acoustic_guitar', 'Applause', 'Bark', 'Bass_drum', 'Burping_or_eructation'],
      dtype='<U21')

In [81]:
tested = [md.test_ds.fnames[i].split('/')[-1] for i in range(len(test_fnames))]; tested[:5]

['c9a6616a.wav',
 'f222ac0b.wav',
 '0e3cdc5d.wav',
 'b983c6d9.wav',
 '778306c2.wav']

In [41]:
idx = []
for fname in test_fnames:
    for name in tested:
        if name == fname:
            idx.append(tested.index(name))

In [95]:
idx = [[tested.index(name) for name in tested if name==fname] for fname in test_fnames]

In [102]:
aud,_ = librosa.load('data/freesound/audio_test/b39975f5.wav', sr=None)


array([], dtype=float32)

In [91]:
idx[:5]

[[2187], [5370], [6500], [7476], [5462]]

In [83]:
[tested[i] for i in idx[:5]]

TypeError: list indices must be integers or slices, not list

In [42]:
[tested[i] for i in idx[:5]]

['00063640.wav',
 '0013a1db.wav',
 '002bb878.wav',
 '002d392d.wav',
 '00326aa9.wav']

In [43]:
test_fnames[:5]

['00063640.wav',
 '0013a1db.wav',
 '002bb878.wav',
 '002d392d.wav',
 '00326aa9.wav']

In [44]:
test['label'] = [preds[i] for i in idx]

In [45]:
test.head()

fname                                     label
0  00063640.wav  Fireworks Finger_snapping Microwave_oven
1  0013a1db.wav                        Oboe Flute Trumpet
2  002bb878.wav                 Bass_drum Knock Fireworks
3  002d392d.wav               Double_bass Bass_drum Knock
4  00326aa9.wav             Oboe Electric_piano Telephone

In [46]:
test.to_csv('sub6.csv', index=False)

In [47]:
test.shape

(9400, 2)

In [55]:
submission = pd.read_csv('sub2.csv'); submission.head()

fname                                  label
0  00063640.wav                Cowbell Harmonica Cough
1  0013a1db.wav  Telephone Acoustic_guitar Double_bass
2  002bb878.wav                 Cowbell Oboe Bass_drum
3  002d392d.wav        Laughter Harmonica Glockenspiel
4  00326aa9.wav   Telephone Microwave_oven Double_bass